In [43]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [44]:
# Load the Excel file
file_path = 'Cotton.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# Function to process data
def process_data(df, start_date='2012-01-01', end_date='2017-12-31'):
    df = df.drop_duplicates(subset='date')
    df = df.set_index('date').reindex(pd.date_range(start=start_date, end=end_date)).rename_axis('date').reset_index()
    df['minimum'] = df['minimum'].interpolate().fillna(method='bfill').fillna(method='ffill')
    df['maximum'] = df['maximum'].interpolate().fillna(method='bfill').fillna(method='ffill')
    return df[['date', 'minimum', 'maximum']]

# Process data for Multan
multan_khal_df = data[(data['station_en'] == 'Multan') & (data['by_product_en'] == 'banola_khal')]
multan_khal_df = process_data(multan_khal_df)

# Process data for Burewala
burewala_khal_df = data[(data['station_en'] == 'Burewala') & (data['by_product_en'] == 'banola_khal')]
burewala_khal_df = process_data(burewala_khal_df)

# Split into training and test sets
train_multan = multan_khal_df[multan_khal_df['date'] < '2017-01-01']
test_multan = multan_khal_df[multan_khal_df['date'] >= '2017-01-01']

train_burewala = burewala_khal_df[burewala_khal_df['date'] < '2017-01-01']
test_burewala = burewala_khal_df[burewala_khal_df['date'] >= '2017-01-01']

/var/folders/hq/f1f9rdc54j7957q7yl8hsy7r0000gn/T/ipykernel_29613/1216749193.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['minimum'] = df['minimum'].interpolate().fillna(method='bfill').fillna(method='ffill')
/var/folders/hq/f1f9rdc54j7957q7yl8hsy7r0000gn/T/ipykernel_29613/1216749193.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['maximum'] = df['maximum'].interpolate().fillna(method='bfill').fillna(method='ffill')
/var/folders/hq/f1f9rdc54j7957q7yl8hsy7r0000gn/T/ipykernel_29613/1216749193.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['minimum'] = df['minimum'].interpolate().fillna(method='bfill').fillna(method='ffill')
/var/folders/hq/f1f9rdc54j7957q7yl8hsy7r0000gn/T/ipykernel_29613/1216749193.py

In [58]:
# Function to run Prophet
def run_prophet(train_df, test_df):
    # Prepare the data for Prophet
    train_min = train_df[['date', 'minimum']].rename(columns={'date': 'ds', 'minimum': 'y'})
    test_min = test_df[['date', 'minimum']].rename(columns={'date': 'ds', 'minimum': 'y'})
    train_max = train_df[['date', 'maximum']].rename(columns={'date': 'ds', 'maximum': 'y'})
    test_max = test_df[['date', 'maximum']].rename(columns={'date': 'ds', 'maximum': 'y'})
    
    # Train and predict minimum
    prophet_model_min = Prophet()
    prophet_model_min.fit(train_min)
    future_min = prophet_model_min.make_future_dataframe(periods=len(test_min), freq='D')
    forecast_min = prophet_model_min.predict(future_min)
    
    # Train and predict maximum
    prophet_model_max = Prophet()
    prophet_model_max.fit(train_max)
    future_max = prophet_model_max.make_future_dataframe(periods=len(test_max), freq='D')
    forecast_max = prophet_model_max.predict(future_max)
    
    # Merge predictions with actual values
    forecast_min = forecast_min[['ds', 'yhat']].rename(columns={'yhat': 'Predicted_Minimum'})
    forecast_max = forecast_max[['ds', 'yhat']].rename(columns={'yhat': 'Predicted_Maximum'})
    prophet_forecast = forecast_min.merge(forecast_max, on='ds')
    prophet_forecast = prophet_forecast.merge(test_df, left_on='ds', right_on='date')
    
    # Calculate RMSE
    prophet_rmse_min = mean_squared_error(prophet_forecast['minimum'], prophet_forecast['Predicted_Minimum'], squared=False)
    prophet_rmse_max = mean_squared_error(prophet_forecast['maximum'], prophet_forecast['Predicted_Maximum'], squared=False)
    
    return prophet_forecast[['date', 'minimum', 'Predicted_Minimum', 'maximum', 'Predicted_Maximum']], prophet_rmse_min, prophet_rmse_max

# Function to run ARIMA
def run_arima(train_df, test_df):
    # Fit ARIMA models for minimum and maximum
    arima_model_min = ARIMA(train_df['minimum'], order=(5, 1, 0))
    arima_fit_min = arima_model_min.fit()
    arima_forecast_min = arima_fit_min.forecast(steps=len(test_df))
    
    arima_model_max = ARIMA(train_df['maximum'], order=(5, 1, 0))
    arima_fit_max = arima_model_max.fit()
    arima_forecast_max = arima_fit_max.forecast(steps=len(test_df))
    
    # Combine forecasts with actual values
    arima_forecast = test_df.copy()
    arima_forecast['Predicted_Minimum'] = arima_forecast_min
    arima_forecast['Predicted_Maximum'] = arima_forecast_max
    
    # Calculate RMSE
    arima_rmse_min = mean_squared_error(test_df['minimum'], arima_forecast_min, squared=False)
    arima_rmse_max = mean_squared_error(test_df['maximum'], arima_forecast_max, squared=False)
    
    return arima_forecast[['date', 'minimum', 'Predicted_Minimum', 'maximum', 'Predicted_Maximum']], arima_rmse_min, arima_rmse_max

# Function to run SARIMA
def run_sarima(train_df, test_df):
    # Fit SARIMA models for minimum and maximum
    sarima_model_min = SARIMAX(train_df['minimum'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    sarima_fit_min = sarima_model_min.fit()
    sarima_forecast_min = sarima_fit_min.forecast(steps=len(test_df))
    
    sarima_model_max = SARIMAX(train_df['maximum'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    sarima_fit_max = sarima_model_max.fit()
    sarima_forecast_max = sarima_fit_max.forecast(steps=len(test_df))
    
    # Combine forecasts with actual values
    sarima_forecast = test_df.copy()
    sarima_forecast['Predicted_Minimum'] = sarima_forecast_min
    sarima_forecast['Predicted_Maximum'] = sarima_forecast_max
    
    # Calculate RMSE
    sarima_rmse_min = mean_squared_error(test_df['minimum'], sarima_forecast_min, squared=False)
    sarima_rmse_max = mean_squared_error(test_df['maximum'], sarima_forecast_max, squared=False)
    
    return sarima_forecast[['date', 'minimum', 'Predicted_Minimum', 'maximum', 'Predicted_Maximum']], sarima_rmse_min, sarima_rmse_max

# Function to run LSTM
def run_lstm_final(train_df, test_df, seq_length=30):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train_df[['minimum', 'maximum']])
    scaled_test = scaler.transform(test_df[['minimum', 'maximum']])
    
    def create_sequences(data, seq_length):
        xs, ys_min, ys_max = [], [], []
        for i in range(len(data) - seq_length):
            x = data[i:i+seq_length]
            y_min = data[i+seq_length, 0]
            y_max = data[i+seq_length, 1]
            xs.append(x)
            ys_min.append(y_min)
            ys_max.append(y_max)
        return np.array(xs), np.array(ys_min), np.array(ys_max)
    
    X_train, y_train_min, y_train_max = create_sequences(scaled_train, seq_length)
    X_test, y_test_min, y_test_max = create_sequences(scaled_test, seq_length)
    
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, 2)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(2))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, np.column_stack((y_train_min, y_train_max)), batch_size=1, epochs=1)
    
    # Predictions on the training set
    train_predictions = model.predict(X_train)
    train_predictions = scaler.inverse_transform(train_predictions)
    y_train_combined = np.column_stack((y_train_min, y_train_max))
    y_train_combined = scaler.inverse_transform(y_train_combined)
    y_train_min, y_train_max = y_train_combined[:, 0], y_train_combined[:, 1]
    
    # Calculate training RMSE and R²
    train_rmse_min = mean_squared_error(y_train_min, train_predictions[:, 0], squared=False)
    train_r2_min = r2_score(y_train_min, train_predictions[:, 0])
    train_rmse_max = mean_squared_error(y_train_max, train_predictions[:, 1], squared=False)
    train_r2_max = r2_score(y_train_max, train_predictions[:, 1])
    
    # Predictions on the testing set
    lstm_predictions = model.predict(X_test)
    lstm_predictions = scaler.inverse_transform(lstm_predictions)
    y_test_combined = np.column_stack((y_test_min, y_test_max))
    y_test_combined = scaler.inverse_transform(y_test_combined)
    y_test_min, y_test_max = y_test_combined[:, 0], y_test_combined[:, 1]
    
    # Calculate testing RMSE and R²
    lstm_rmse_min = mean_squared_error(y_test_min, lstm_predictions[:, 0], squared=False)
    lstm_r2_min = r2_score(y_test_min, lstm_predictions[:, 0])
    lstm_rmse_max = mean_squared_error(y_test_max, lstm_predictions[:, 1], squared=False)
    lstm_r2_max = r2_score(y_test_max, lstm_predictions[:, 1])
    
    result_df = test_df[seq_length:].copy()
    result_df['Predicted_Minimum'] = lstm_predictions[:, 0]
    result_df['Actual_Minimum'] = y_test_min
    result_df['Predicted_Maximum'] = lstm_predictions[:, 1]
    result_df['Actual_Maximum'] = y_test_max
    
    return result_df[['date', 'Actual_Minimum', 'Predicted_Minimum', 'Actual_Maximum', 'Predicted_Maximum']], train_rmse_min, train_r2_min, lstm_rmse_min, lstm_r2_min, train_rmse_max, train_r2_max, lstm_rmse_max, lstm_r2_max


In [59]:
# Run and get predictions for Multan
multan_prophet_results, multan_prophet_rmse_min, multan_prophet_rmse_max = run_prophet(train_multan, test_multan)
multan_arima_results, multan_arima_rmse_min, multan_arima_rmse_max = run_arima(train_multan, test_multan)
multan_sarima_results, multan_sarima_rmse_min, multan_sarima_rmse_max = run_sarima(train_multan, test_multan)
multan_lstm_results, multan_train_rmse_min, multan_train_r2_min, multan_rmse_min, multan_r2_min, multan_train_rmse_max, multan_train_r2_max, multan_rmse_max, multan_r2_max = run_lstm_final(train_multan, test_multan)

print(f'Prophet RMSE for Multan Minimum: {multan_prophet_rmse_min}')
print(f'Prophet RMSE for Multan Maximum: {multan_prophet_rmse_max}')
print(f'ARIMA RMSE for Multan Minimum: {multan_arima_rmse_min}')
print(f'ARIMA RMSE for Multan Maximum: {multan_arima_rmse_max}')
print(f'SARIMA RMSE for Multan Minimum: {multan_sarima_rmse_min}')
print(f'SARIMA RMSE for Multan Maximum: {multan_sarima_rmse_max}')
print()
print()
print(f'LSTM Train RMSE for Multan Minimum: {multan_train_rmse_min}')
print(f'LSTM Train R² for Multan Minimum: {multan_train_r2_min}')
print(f'LSTM Test RMSE for Multan Minimum: {multan_rmse_min}')
print(f'LSTM Test R² for Multan Minimum: {multan_r2_min}')
print(f'LSTM Train RMSE for Multan Maximum: {multan_train_rmse_max}')
print(f'LSTM Train R² for Multan Maximum: {multan_train_r2_max}')
print(f'LSTM Test RMSE for Multan Maximum: {multan_rmse_max}')
print(f'LSTM Test R² for Multan Maximum: {multan_r2_max}')


16:27:55 - cmdstanpy - INFO - Chain [1] start processing
16:27:56 - cmdstanpy - INFO - Chain [1] done processing
16:27:56 - cmdstanpy - INFO - Chain [1] start processing
16:27:57 - cmdstanpy - INFO - Chain [1] done processing
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.92068D+00    |proj g|=  6.31409D-02

At iterate    5    f=  4.88373D+00    |proj g|=  1.63377D-02

At iterate   10    f=  4.78357D+00    |proj g|=  1.36334D-02

At iterate   15    f=  4.78287D+00    |proj g|=  3.93725D-04

At iterate   20    f=  4.78285D+00    |proj g|=  7.39025D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     23     26      1     0     0   4.153D-06   4.783D+00
  F =   4.7828464821455619     

CONVERG

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.86799D+00    |proj g|=  5.77541D-02

At iterate    5    f=  4.83468D+00    |proj g|=  2.29137D-02

At iterate   10    f=  4.75032D+00    |proj g|=  6.12774D-03

At iterate   15    f=  4.74820D+00    |proj g|=  1.16556D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     17     32      1     0     0   1.046D-05   4.748D+00
  F =   4.7481960858160308     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1797/1797 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.0093
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Prophet RMSE for Multan Minimum: 90.69536554157031
Prophet RMSE for Multan Maximum: 75.8215463331507
ARIMA RMSE for Multan Minimum: 130.5266359868141
ARIMA RMSE for Multan Maximum: 130.44992582706615
SARIMA RMSE for Multan Minimum: 180.44562625567903
SARIMA RMSE for Multan Maximum: 174.49982982112155


LSTM Train RMSE for Multan Minimum: 46.648069626525995
LSTM Train R² for Multan Minimum: 0.9795359375515024
LSTM Test RMSE for Multan Minimum: 37.36439937469589
LSTM Test R² for Multan Minimum: 0.9244971830859178
LSTM Train RMSE for Multan Maximum: 40.59101275453381
LSTM Train R² for Multan Maximum: 0.9839613059254293
LSTM Test RMSE for Multan Maximum: 35.27344371298464
LSTM Test R² for Multan Maximum: 0.9323299597243487


/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the 

In [61]:
# Run and get predictions for Burewala
burewala_prophet_results, burewala_prophet_rmse_min, burewala_prophet_rmse_max = run_prophet(train_burewala, test_burewala)
burewala_arima_results, burewala_arima_rmse_min, burewala_arima_rmse_max = run_arima(train_burewala, test_burewala)
burewala_sarima_results, burewala_sarima_rmse_min, burewala_sarima_rmse_max = run_sarima(train_burewala, test_burewala)
burewala_lstm_results, burewala_train_rmse_min, burewala_train_r2_min, burewala_rmse_min, burewala_r2_min, burewala_train_rmse_max, burewala_train_r2_max, burewala_rmse_max, burewala_r2_max = run_lstm_final(train_burewala, test_burewala)

print(f'Prophet RMSE for Burewala Minimum: {burewala_prophet_rmse_min}')
print(f'Prophet RMSE for Burewala Maximum: {burewala_prophet_rmse_max}')
print(f'ARIMA RMSE for Burewala Minimum: {burewala_arima_rmse_min}')
print(f'ARIMA RMSE for Burewala Maximum: {burewala_arima_rmse_max}')
print(f'SARIMA RMSE for Burewala Minimum: {burewala_sarima_rmse_min}')
print(f'SARIMA RMSE for Burewala Maximum: {burewala_sarima_rmse_max}')
print()
print()
print(f'LSTM Train RMSE for Burewala Minimum: {burewala_train_rmse_min}')
print(f'LSTM Train R² for Burewala Minimum: {burewala_train_r2_min}')
print(f'LSTM Test RMSE for Burewala Minimum: {burewala_rmse_min}')
print(f'LSTM Test R² for Burewala Minimum: {burewala_r2_min}')
print(f'LSTM Train RMSE for Burewala Maximum: {burewala_train_rmse_max}')
print(f'LSTM Train R² for Burewala Maximum: {burewala_train_r2_max}')
print(f'LSTM Test RMSE for Burewala Maximum: {burewala_rmse_max}')
print(f'LSTM Test R² for Burewala Maximum: {burewala_r2_max}')


16:29:08 - cmdstanpy - INFO - Chain [1] start processing
16:29:09 - cmdstanpy - INFO - Chain [1] done processing
16:29:09 - cmdstanpy - INFO - Chain [1] start processing
16:29:09 - cmdstanpy - INFO - Chain [1] done processing
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.19630D+00    |proj g|=  6.26386D-02


 This problem is unconstrained.



At iterate    5    f=  5.16068D+00    |proj g|=  1.39272D-02

At iterate   10    f=  5.06684D+00    |proj g|=  1.47377D-02

At iterate   15    f=  5.06535D+00    |proj g|=  2.47323D-03

At iterate   20    f=  5.06491D+00    |proj g|=  2.85123D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     21     27      1     0     0   2.406D-06   5.065D+00
  F =   5.0649130034083587     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.19826D+00    |proj g|=  6.39549D-02

At iterate    5    f=  5.16975D+00    |proj g|=  9.79096D-03

At iterate   10    f=  5.08141D+00    |proj g|=  1.03741D-02

At iterate   15    f=  5.07706D+00    |proj g|=  6.26074D-02

At iterate   20    f=  5.07136D+00    |proj g|=  1.19131D-03

At iterate   25    f=  5.07135D+00    |proj g|=  2.21931D-03

At iterate   30    f=  5.07133D+00    |proj g|=  2.10130D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nac

/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1797/1797 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.0056
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Prophet RMSE for Burewala Minimum: 117.83649745316451
Prophet RMSE for Burewala Maximum: 114.6232296677058
ARIMA RMSE for Burewala Minimum: 150.40749973813266
ARIMA RMSE for Burewala Maximum: 147.86875179342786
SARIMA RMSE for Burewala Minimum: 222.1142673158194
SARIMA RMSE for Burewala Maximum: 212.53018793881697


LSTM Train RMSE for Burewala Minimum: 52.44851921326568
LSTM Train R² for Burewala Minimum: 0.9748551141828377
LSTM Test RMSE for Burewala Minimum: 36.983797026977214
LSTM Test R² for Burewala Minimum: 0.9403964416474401
LSTM Train RMSE for Burewala Maximum: 49.26742316482881
LSTM Train R² for Burewala Maximum: 0.9763044350072134
LSTM Test RMSE for Burewala Maximum: 38.1821193692004
LSTM Test R² for Burewala Maximum: 0.9361078348650845


/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the 

In [62]:
# Save results to Excel
with pd.ExcelWriter('Cotton_Predictions.xlsx') as writer:
    multan_lstm_results.to_excel(writer, sheet_name='Multan_LSTM', index=False)
    burewala_lstm_results.to_excel(writer, sheet_name='Burewala_LSTM', index=False)

print("Predictions saved to Cotton_Predictions.xlsx")

Predictions saved to Cotton_Predictions.xlsx
